In [3]:
# Goal: Analyze music data to see how successful models are at predicting song danceability
# Predictors: instrumentalness, tempo, temporal_features0-223
# Data Source: https://github.com/mdeff/fma 
#    Subset of data with 13,133 songs
#
# Part 2: Training and Testing Models
#    Multivariate Linear Regression
#    RandomForest
#    Multilayer Perceptron
#    Boosting
# Date last updated: 12/21/2023
# Author: Kay Rubio

import joblib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPRegressor
import time
# ignore future warnings that aren't necessary for this project
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [17]:
# import data built during data preparation (see separate notebook)
x_train = pd.read_csv("./x_train.csv")
y_train = pd.read_csv("./y_train.csv")
x_test = pd.read_csv("./x_test.csv")
y_test = pd.read_csv("./y_test.csv")
# trimmed data for regression model only
x_train_reg2 = pd.read_csv("./x_train_reg2.csv")
x_test_reg2 = pd.read_csv("./x_test_reg2.csv")
# further trimmed data for regression model only
x_train_reg3 = pd.read_csv("./x_train_reg3.csv")
x_test_reg3 = pd.read_csv("./x_test_reg3.csv")

In [18]:
# drop the ID columns from the datasets
x_train.drop('Unnamed: 0', axis=1, inplace=True) 
y_train.drop('Unnamed: 0', axis=1, inplace=True) 
x_test.drop('Unnamed: 0', axis=1, inplace=True) 
y_test.drop('Unnamed: 0', axis=1, inplace=True) 
x_train_reg2.drop('Unnamed: 0', axis=1, inplace=True) 
x_test_reg2.drop('Unnamed: 0', axis=1, inplace=True)
x_train_reg3.drop('Unnamed: 0', axis=1, inplace=True) 
x_test_reg3.drop('Unnamed: 0', axis=1, inplace=True)
x_test_reg2.head()

,temporal_features98,temporal_features99,temporal_features101,temporal_features110,temporal_features111,temporal_features113,temporal_features123,temporal_features125,temporal_features127,temporal_features129,...,temporal_features163,temporal_features165,temporal_features166,temporal_features183,temporal_features185,temporal_features200,temporal_features201,temporal_features214,temporal_features216,temporal_features217
0,-5.366536,-7.917288,-29.483248,-10.969000,-12.683000,-31.177999,1745.207520,857.857544,450.171478,239.780121,...,159.458008,123.761002,125.893005,0.439256,3.663183,0.060203,0.043990,-0.284468,0.272164,0.200090
1,17.757927,-6.526848,-2.927552,21.014500,-5.910500,-2.620500,481.618347,438.531799,173.483246,119.562477,...,187.322998,158.572998,143.895996,11.812191,2.641223,0.059045,0.046935,-2.943317,0.286168,0.219070
2,-24.476557,-14.597180,-30.570845,-19.882500,-14.056499,-32.332001,1041.254639,601.637512,616.665955,412.457336,...,237.859009,168.804993,160.173004,5.439683,2.165459,0.049192,0.032995,-2.471529,0.244255,0.220865
3,17.759216,-10.643620,-25.965639,20.854000,-9.445499,-28.880001,316.353668,314.000122,99.852158,100.579712,...,111.977997,81.070999,61.323997,3.007475,3.836738,0.068552,0.035010,-2.516262,0.390579,0.275075
4,56.503654,9.047318,-9.762955,56.953999,4.429500,-15.351999,2217.191895,1086.543945,420.112000,331.661926,...,190.218994,165.492004,184.391998,14.066206,12.843975,0.059838,0.045315,-2.203966,0.272245,0.208930


In [5]:
# Model #1: Multivariate Linear Regression 
#
# Version #1 using 27 temporal features, all with a correlation to danceability that is at least 0.3
# but some predictors aren't normally distributed, have poor homoscedasticity with danceability, or lots of 
# multivariate outliers

In [10]:
# Learn some information about the model
print(help(LinearRegression))

Help on class LinearRegression in module sklearn.linear_model._base:

class LinearRegression(sklearn.base.MultiOutputMixin, sklearn.base.RegressorMixin, LinearModel)
 |  LinearRegression(*, fit_intercept=True, copy_X=True, n_jobs=None, positive=False)
 |  
 |  Ordinary least squares Linear Regression.
 |  
 |  LinearRegression fits a linear model with coefficients w = (w1, ..., wp)
 |  to minimize the residual sum of squares between the observed targets in
 |  the dataset, and the targets predicted by the linear approximation.
 |  
 |  Parameters
 |  ----------
 |  fit_intercept : bool, default=True
 |      Whether to calculate the intercept for this model. If set
 |      to False, no intercept will be used in calculations
 |      (i.e. data is expected to be centered).
 |  
 |  copy_X : bool, default=True
 |      If True, X will be copied; else, it may be overwritten.
 |  
 |  n_jobs : int, default=None
 |      The number of jobs to use for the computation. This will only provide
 |  

In [19]:
# fit model on the training data
reg_model = LinearRegression().fit(x_train_reg2, y_train)

In [20]:
# will give you an array holding the y-intercept of regression line
reg_model.intercept_

array([0.55159423])

In [21]:
# print the coefficients for each predictor, in the order that they appear in the training data
reg_model.coef_

array([[-1.45284300e-03,  2.49173269e-03, -2.20370662e-05,
        -5.68165904e-04,  8.15157717e-04,  2.85407819e-03,
        -5.20622340e-06,  1.59983128e-05, -6.13149026e-05,
         1.83624973e-04, -1.72962886e-06,  9.76786631e-05,
        -4.70457901e-05, -3.02939623e-04,  1.49319875e-04,
        -2.49358322e-04,  9.83725977e-05,  5.41487304e-05,
        -4.60682061e-04,  2.50924929e-04, -5.76516729e-04,
        -1.22380283e-03,  2.50310699e-01,  1.53355725e-01,
        -4.01923455e-03, -4.27627123e-01,  2.15568282e-01]])

In [ ]:
# Most of these regression coefficients are less than .05, indicating significant p-values, meaning most
# predictors are significant in the model as a whole. Further refinement like step-wise regression could be done
# to remove the few that aren't significant, but probably not necessary since I'll be running some more powerful models
# down below like random forest.

In [22]:
# If we were interested in seeing exactly which predictors were/weren't significant in the model, we could match the
# array above with the column names to investigate further.
column_names = x_train_reg2.columns.tolist()
print("Column Names:", column_names)

Column Names: ['temporal_features98', 'temporal_features99', 'temporal_features101', 'temporal_features110', 'temporal_features111', 'temporal_features113', 'temporal_features123', 'temporal_features125', 'temporal_features127', 'temporal_features129', 'temporal_features130', 'temporal_features134', 'temporal_features139', 'temporal_features141', 'temporal_features149', 'temporal_features154', 'temporal_features161', 'temporal_features163', 'temporal_features165', 'temporal_features166', 'temporal_features183', 'temporal_features185', 'temporal_features200', 'temporal_features201', 'temporal_features214', 'temporal_features216', 'temporal_features217']


In [112]:
reg_model.score(x_test_reg2, y_test) # Calculate R2

0.5993441649927478

In [ ]:
# R-squared indicates that almost 60% of the variability in danceability can be explained by these temporal_features
# which isn't bad for a linear model on kinda messy data

In [113]:
y_pred = reg_model.predict(x_test_reg2) # get the predicted y-values

In [114]:
print(y_pred)

[[0.43818462]
 [0.47774869]
 [0.47952883]
 ...
 [0.36269824]
 [0.43826685]
 [0.6187952 ]]


In [115]:
# Check the mean absolute error (MAE) comparing the model's predicted outcomes with the actual outcomes
mean_absolute_error(y_test, y_pred)

0.09947146084610314

In [ ]:
# Model's predictions are above/below the real outcome value by an avg of ~0.0995. Since danceability ranges from 0-1, that's not too bad

In [ ]:
# Version #2 using 15 temporal features, all with a correlation to danceability that is at least 0.3
# and all with somewhat better distributions and homoscedasticity with danceability

In [118]:
# fit the the model on training data
reg_model2 = LinearRegression().fit(x_train_reg3, y_train)
# Check y-intercept and coefficients of each predictor
reg_model2.intercept_
reg_model2.coef_

array([[-1.47234479e-03,  2.51936425e-03,  1.05799318e-03,
        -8.39157778e-04,  1.37481307e-03,  3.03579692e-03,
         1.06125685e-04, -3.39884753e-04, -6.31195384e-04,
         3.70165265e-05, -3.23145539e-04,  9.83081232e-05,
        -9.24754366e-05, -2.98982465e-04,  3.54915743e-04]])

In [119]:
reg_model2.score(x_test_reg3, y_test) # Calculate R2

0.5579600307506984

In [ ]:
# Slightly worse R-squared from version #1, 56% of the variability in danceability can be explained by these temporal_features

In [122]:
# get the predicted y-values
y_pred = reg_model2.predict(x_test_reg3)
print(y_pred)

[[0.41286885]
 [0.49285015]
 [0.4505156 ]
 ...
 [0.38253946]
 [0.43973822]
 [0.60906937]]


In [123]:
mean_absolute_error(y_test, y_pred)

0.10578788198731864

In [ ]:
# Version 2 of multiple regression's predicted danceability scores are above/below the real values by an avg of ~0.11. 
# Since danceability ranges from 0-1, that's a little worse than the first regression model, as expected given the 
# slightly worse R-squared

In [ ]:
# Model #2: RandomForestRegressor 
#    Version 1: Without hyperparameter tuning

In [138]:
# check information about the model and its hyperparameters
print(help(RandomForestRegressor))

Help on class RandomForestRegressor in module sklearn.ensemble._forest:

class RandomForestRegressor(ForestRegressor)
 |  RandomForestRegressor(n_estimators=100, *, criterion='squared_error', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=1.0, max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, ccp_alpha=0.0, max_samples=None)
 |  
 |  A random forest regressor.
 |  
 |  A random forest is a meta estimator that fits a number of classifying
 |  decision trees on various sub-samples of the dataset and uses averaging
 |  to improve the predictive accuracy and control over-fitting.
 |  The sub-sample size is controlled with the `max_samples` parameter if
 |  `bootstrap=True` (default), otherwise the whole dataset is used to build
 |  each tree.
 |  
 |  For a comparison between tree-based ensemble models see the example
 |  :ref:`sphx_glr_auto_examp

In [127]:
rr = RandomForestRegressor()

In [128]:
# Fit the model using the training data Took ~3 min on my laptop
rr.fit(x_train, y_train.values.ravel())

RandomForestRegressor()

In [129]:
# Run cross-validation, which means split the dataset into pieces, and iterate over it 
# using a different piece as test and rest as train
# This took about 15 minutes to run on my laptop
scores = cross_val_score(rr, x_train, y_train.values.ravel(), cv=5)

In [130]:
scores

array([0.76016115, 0.73282828, 0.75136312, 0.7371274 , 0.74669975])

In [ ]:
# Model is supposedly between 73-76% accurate when trained on 4/5 of the training data, and tested on the other 1/5

In [135]:
# Get the predicted y-values
y_pred = rr.predict(x_test)
print(y_pred)

[0.48602867 0.43243468 0.47072673 ... 0.31505692 0.44021106 0.6864622 ]


In [136]:
# Will give you MAE based on comparing the actual outcomes with predicted outcomes
mean_absolute_error(y_test, y_pred)

0.07727108712718854

In [ ]:
# Model's predicted danceability scores on test data are above/below the real danceability scores by an avg of ~0.0773. 
# Since danceability ranges from 0-1, that's pretty good, stronger than regression

In [ ]:
# Version 2: With hyperparameter tuning
#    model above uses these default hyperparameters:
#      n_estimators: (number of trees) default=100
#      max_depth: (max depth of each tree) default=None

# Model below will try these alternate hyperparameter settings
#   n_estimators: [5, 50, 100]
#   max_depth: [2, 10, 20, None]

In [140]:
# Define a function to help print results of each model
def print_results(results):
	print('BEST PARAMS: {}\n'.format(results.best_params_))
	means = results.cv_results_['mean_test_score']
	stds = results.cv_results_['std_test_score']
	for mean, std, params in zip(means, stds, results.cv_results_['params']):
		print('{} (+/-{}) for {}'.format(round(mean, 3), round(std * 2, 3), params))

In [157]:
# Define a collection of parameters for hypertuning
parameters = { 
    'n_estimators': [5, 50, 100],
    'max_depth': [2, 10, 20, None]
}

In [158]:
# Save the model
rr2 = RandomForestRegressor()

In [153]:
# Run cross-validation which will split the dataset into 5 pieces, and iterate over it using a different piece as test and rest as train.
# It will include the parameters to be hypertuned, and produce 3X4=12 models based on the hyperparameter combos
cv = GridSearchCV(rr2, parameters, cv=5)

In [159]:
# Use cross-validation to fit the model
# Took about 50 min to run on my laptop
cv.fit(x_train, y_train.values.ravel())

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 10, 20, None],
                         'n_estimators': [5, 50, 100]})

In [160]:
print_results(cv)

BEST PARAMS: {'max_depth': None, 'n_estimators': 100}

0.522 (+/-0.023) for {'max_depth': 2, 'n_estimators': 5}
0.533 (+/-0.017) for {'max_depth': 2, 'n_estimators': 50}
0.532 (+/-0.021) for {'max_depth': 2, 'n_estimators': 100}
0.699 (+/-0.024) for {'max_depth': 10, 'n_estimators': 5}
0.738 (+/-0.02) for {'max_depth': 10, 'n_estimators': 50}
0.74 (+/-0.021) for {'max_depth': 10, 'n_estimators': 100}
0.684 (+/-0.029) for {'max_depth': 20, 'n_estimators': 5}
0.743 (+/-0.021) for {'max_depth': 20, 'n_estimators': 50}
0.745 (+/-0.022) for {'max_depth': 20, 'n_estimators': 100}
0.678 (+/-0.026) for {'max_depth': None, 'n_estimators': 5}
0.743 (+/-0.02) for {'max_depth': None, 'n_estimators': 50}
0.746 (+/-0.019) for {'max_depth': None, 'n_estimators': 100}


In [ ]:
# Best model uses {max_depth: None, n_estimators: 10} with accuracy of about 74.6%
# Second best model uses {max_depth: 20, n_estimators: 100} with accuracy of about 74.5%
# Third best model uses {max_depth: None, n_estimators: 50} with accuracy of about 74.4% and lower SD

In [167]:
# Pick the 3 best sets of hyperparameters to create individual models
rr1 = RandomForestRegressor(max_depth=None, n_estimators=100)
rr2 = RandomForestRegressor(max_depth=20, n_estimators=100)
rr3 = RandomForestRegressor(max_depth=None, n_estimators=50)

In [168]:
# Train models on training data
rr1.fit(x_train, y_train.values.ravel())

RandomForestRegressor()

In [170]:
rr2.fit(x_train, y_train.values.ravel())

RandomForestRegressor(max_depth=20)

In [171]:
rr3.fit(x_train, y_train.values.ravel())

RandomForestRegressor(n_estimators=50)

In [186]:
# Check how good these models are on the training data. It's possible the best model was overfitting
# and might not work as well on the training data
for mdl in [rr1, rr2, rr3]:
    y_pred = mdl.predict(x_test) # get the predicted y-values
    print(mean_absolute_error(y_test, y_pred)) # print the mean difference between predicted and actual danceability score

0.07734363446332014
0.0772812648282704
0.07771370618324094


In [ ]:
# Here, all 3 models are performing extremely simiarly, with their predicted danceability score only an avg of ~0.0773 pts away
# from the real score. The second one performs slightly better on the test data than the others, so we'll assume this is the best

In [187]:
# pickle best model
joblib.dump(rr2, './RandomForestRegressor_model.pkl')

['./RandomForestRegressor_model.pkl']

In [ ]:
# Model #3: Multilayer Perceptron Regression Model
#    Version 1: Without hyperparameter tuning

In [193]:
# print some information about the model and its hyperparameters
print(help(MLPRegressor))

Help on class MLPRegressor in module sklearn.neural_network._multilayer_perceptron:

class MLPRegressor(sklearn.base.RegressorMixin, BaseMultilayerPerceptron)
 |  MLPRegressor(hidden_layer_sizes=(100,), activation='relu', *, solver='adam', alpha=0.0001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10, max_fun=15000)
 |  
 |  Multi-layer Perceptron regressor.
 |  
 |  This model optimizes the squared error using LBFGS or stochastic gradient
 |  descent.
 |  
 |  .. versionadded:: 0.18
 |  
 |  Parameters
 |  ----------
 |  hidden_layer_sizes : array-like of shape(n_layers - 2,), default=(100,)
 |      The ith element represents the number of neurons in the ith
 |      hidden layer.
 |  
 |  activation : {'identity', 

In [208]:
# Save the model with some hyperparameters
mlp = MLPRegressor(random_state=1, max_iter=500)

In [209]:
# Fit the model to the training data
# Did not take long on my laptop
mlp.fit(x_train, y_train.values.ravel())

MLPRegressor(max_iter=500, random_state=1)

In [211]:
# Use the model to predict danceability scores in the test data
y_pred = mlp.predict(x_test)
print(y_pred)

[0.80079934 1.65063427 0.52247218 ... 0.49280228 1.44024457 0.90245539]


In [ ]:
# This isn't looking good, the outcome should not be above 1, and some of these are too high

In [212]:
# Calculate the mean difference between predicted and actual danceability in the test data
mean_absolute_error(y_test, y_pred)
print(mean_absolute_error(y_test, y_pred))

1.322998348519081


In [ ]:
# Avg predicted score is 1.32 points away from the actual score. That's worse than regression and random forest
# This may be expected because multilayer perceptron performs better when there's massive amounts of data and 
# there are only 10,000 songs in the training data

In [216]:
# Version #2: Multilayer Perceptron Regression Model with hyperparameter tuning

In [217]:
# hidden_layer_sizes: an array of tuples, with first setting is number of nodes and second setting (left blank here for default) 
# is number of layers

# activation: different activation functionality for the hidden layer

parameters = {
	'hidden_layer_sizes': [(10,), (50,), (100,)],
	'activation': ['relu', 'tanh', 'logistic']
}

In [219]:
# Make the model and set up grid serach cross validation with the hyperparameter settings
mlp2 = MLPRegressor()
cv = GridSearchCV(mlp2, parameters, cv=5)

In [220]:
# Fit all the models with various hyperparameter settings
# Took only a few min to run on my laptop
cv.fit(x_train, y_train.values.ravel())

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


GridSearchCV(cv=5, estimator=MLPRegressor(),
             param_grid={'activation': ['relu', 'tanh', 'logistic'],
                         'hidden_layer_sizes': [(10,), (50,), (100,)]})

In [221]:
print_results(cv)

BEST PARAMS: {'activation': 'logistic', 'hidden_layer_sizes': (50,)}

-376.495 (+/-971.145) for {'activation': 'relu', 'hidden_layer_sizes': (10,)}
-768.948 (+/-1640.969) for {'activation': 'relu', 'hidden_layer_sizes': (50,)}
-698.479 (+/-407.968) for {'activation': 'relu', 'hidden_layer_sizes': (100,)}
0.136 (+/-0.634) for {'activation': 'tanh', 'hidden_layer_sizes': (10,)}
0.518 (+/-0.039) for {'activation': 'tanh', 'hidden_layer_sizes': (50,)}
0.497 (+/-0.056) for {'activation': 'tanh', 'hidden_layer_sizes': (100,)}
0.579 (+/-0.06) for {'activation': 'logistic', 'hidden_layer_sizes': (10,)}
0.627 (+/-0.039) for {'activation': 'logistic', 'hidden_layer_sizes': (50,)}
0.619 (+/-0.049) for {'activation': 'logistic', 'hidden_layer_sizes': (100,)}


In [ ]:
# activation: relu has negative accuracy? 
# The rest have poorer accuracy than random forest, and at best reach only 63%
# at best reaching 63% accuracy with these hyperparameters: {'activation': 'logistic', 'hidden_layer_sizes': (50,)}
# This may be expected, as multilayer perceptron needs a ton of data, and the training data has less than 10,000 rows

In [ ]:
# Model #4: Boosting Gradient Trees
# Trees that learn from each other. Take longer to train than random forest, but fast to make predictions

In [226]:
# Print some info about it and it's hyperparameters
print(help(GradientBoostingRegressor))

Help on class GradientBoostingRegressor in module sklearn.ensemble._gb:

class GradientBoostingRegressor(sklearn.base.RegressorMixin, BaseGradientBoosting)
 |  GradientBoostingRegressor(*, loss='squared_error', learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, init=None, random_state=None, max_features=None, alpha=0.9, verbose=0, max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)
 |  
 |  Gradient Boosting for regression.
 |  
 |  This estimator builds an additive model in a forward stage-wise fashion; it
 |  allows for the optimization of arbitrary differentiable loss functions. In
 |  each stage a regression tree is fit on the negative gradient of the given
 |  loss function.
 |  
 |  :class:`sklearn.ensemble.HistGradientBoostingRegressor` is a much faster
 |  variant of this algorithm

In [23]:
# save the model
gb = GradientBoostingRegressor()

In [228]:
# Fit the model to the training data. Took only took a few min to run on my laptop
gb.fit(x_train, y_train.values.ravel())

In [230]:
# Run model on test data to get the predicted danceability scores
y_pred = gb.predict(x_test)
print(y_pred)

[0.45685882 0.41726136 0.48079732 ... 0.22242709 0.38249673 0.71868298]


In [231]:
# Calculate mean absolute error between predicted danceability and actual danceability in the test data
mean_absolute_error(y_test, y_pred)
print(mean_absolute_error(y_test, y_pred))

0.07277560281870248


In [232]:
# This is pretty good accuracy. The model's predictions of danceability were only an avg of 0.0728 pts away from the actual
# danceability which is slightly better than the best random forest model

In [233]:
# Lets try some hyperparameter tuning
# n_estimators is the number of stages or trees. default is 100
# max_depth is number of nodes in a tree. Default is 3
parameters = {
	'n_estimators': [50, 100, 250, 500],
	'max_depth': [3, 7, 9, None],
}

In [234]:
# set up cross validation with the hyperparameter tuning. Will create 4x4=16 models using all possible
# combos of the hyperparameters, and use 4/5 of training data for train and the other 1/5 to test
cv = GridSearchCV(gb, parameters, cv=5)

In [235]:
# Fit the model to the training data
# This took overnight on my laptop.  Kernel went idle after 3 hours, but the notebook cell didn't finish executing 
# till the next morning. Should time it if I ever run it again
cv.fit(x_train, y_train.values.ravel())

GridSearchCV(cv=5, estimator=GradientBoostingRegressor(),
             param_grid={'max_depth': [3, 7, 9, None],
                         'n_estimators': [50, 100, 250, 500]})

In [237]:
print_results(cv)

BEST PARAMS: {'max_depth': 3, 'n_estimators': 500}

0.747 (+/-0.018) for {'max_depth': 3, 'n_estimators': 50}
0.774 (+/-0.017) for {'max_depth': 3, 'n_estimators': 100}
0.793 (+/-0.014) for {'max_depth': 3, 'n_estimators': 250}
0.8 (+/-0.013) for {'max_depth': 3, 'n_estimators': 500}
0.776 (+/-0.01) for {'max_depth': 7, 'n_estimators': 50}
0.784 (+/-0.009) for {'max_depth': 7, 'n_estimators': 100}
0.786 (+/-0.01) for {'max_depth': 7, 'n_estimators': 250}
0.787 (+/-0.009) for {'max_depth': 7, 'n_estimators': 500}
0.764 (+/-0.014) for {'max_depth': 9, 'n_estimators': 50}
0.769 (+/-0.012) for {'max_depth': 9, 'n_estimators': 100}
0.77 (+/-0.013) for {'max_depth': 9, 'n_estimators': 250}
0.77 (+/-0.013) for {'max_depth': 9, 'n_estimators': 500}
0.498 (+/-0.028) for {'max_depth': None, 'n_estimators': 50}
0.495 (+/-0.031) for {'max_depth': None, 'n_estimators': 100}
0.495 (+/-0.026) for {'max_depth': None, 'n_estimators': 250}
0.494 (+/-0.029) for {'max_depth': None, 'n_estimators': 500}


In [238]:
# Best model is: {'max_depth': 3, 'n_estimators': 500} with accuracy of about 80%
# Second best model is: {'max_depth': 3, 'n_estimators': 250} with accuracy of about 79%
# Third best model is: {'max_depth': 7, 'n_estimators': 500} with accuracy of about 79%

# of the models with 100 trees or less, the best is:
# {'max_depth': 7, 'n_estimators': 100} with 78% accuracy

In [24]:
# Pick the 2 best sets of hyperparameters, and the third smaller model with good accuracy but fewer trees
# Fewer trees will mean faster to run, since these trees build upon each other and therefore are built sequentially
gb1 = GradientBoostingRegressor(max_depth=3, n_estimators=500)
gb2 = GradientBoostingRegressor(max_depth=3, n_estimators=250)
gb3 = GradientBoostingRegressor(max_depth=7, n_estimators=100)

In [25]:
# Train models on training data.  Time the one with 500 trees.
start = time.time()
gb1.fit(x_train, y_train.values.ravel())
end = time.time()
model_name_fit_or_predict_time = (end - start) # will return number of seconds it takes

In [249]:
print('This model took about ', round(model_name_fit_or_predict_time / 60, 2), 'minutes to run')

This model took about  4.33 minutes to run


In [242]:
# Train model on training data
gb2.fit(x_train, y_train.values.ravel())

GradientBoostingRegressor(n_estimators=250)

In [243]:
# Train model on training data
gb3.fit(x_train, y_train.values.ravel())

GradientBoostingRegressor(max_depth=7)

In [244]:
# Check how good these models are on the training data. It's possible the best model was overfitting
# and might not work as well on the training data
for mdl in [gb1, gb2, gb3]:
    y_pred = mdl.predict(x_test) # get the predicted y-values
    print(mean_absolute_error(y_test, y_pred))

0.06661576492829122
0.0686302551720496
0.06888456269205578


In [ ]:
# These are great models!  The best, with 500 trees, predicts danceability and is only off the actual score by 0.0666 points

In [245]:
# Pickle best model
joblib.dump(gb1, './GradientBoosterRegressor_model.pkl')

['./GradientBoosterRegressor_model.pkl']

In [ ]:
# Conclusion
# I wanted to use my best model to predict the danceability of some brand new popular songs using their temporal
# features, but alas, spotify no longer provides this data on songs. So, instead, lets simulate using this model
# by selecting a couple of songs from the training data that we know are high, low, and medium in danceability
# and see what the model says about them

# I happen to know that song #8230 Wooden Ships is not very danceable with a score of 0.05166771
# while song #11564 Shakkei (Remixed) by Origamibiro is medium danceable with a score of 0.44688061
# and song #9437 Niris by Nicky Cook is supposed to be super danceable with a score of 0.94879937
# But lets see what the model says

In [26]:
# read it in again with the id's
x_test = pd.read_csv("./x_test.csv")
x_test_3_songs = x_test[x_test['Unnamed: 0'].isin([8230, 11564, 9437])]
print(x_test_3_songs)

      Unnamed: 0  instrumentalness    tempo  temporal_features0  \
246         8230          0.750180  234.501            0.208941   
2135       11564          0.779940   93.495            0.265258   
2533        9437          0.519803  124.035            0.431988   

      temporal_features1  temporal_features2  temporal_features3  \
246             0.199116            0.165297            0.119200   
2135            0.622987            0.496264            0.125611   
2533            0.444268            0.396486            0.235402   

      temporal_features4  temporal_features5  temporal_features6  ...  \
246             0.228610            0.145534            0.215040  ...   
2135            0.320337            0.070170            0.228531  ...   
2533            0.294019            0.271680            0.205089  ...   

      temporal_features214  temporal_features215  temporal_features216  \
246              -3.221379              9.918351              0.685443   
2135             

In [27]:
x_test_3_songs.drop('Unnamed: 0', axis=1, inplace=True)

/var/folders/ct/78f3t3tx0wd8rkf818c7rxd00000gp/T/ipykernel_23743/3679984822.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test_3_songs.drop('Unnamed: 0', axis=1, inplace=True)


In [28]:
x_test_3_songs.head()

,instrumentalness,tempo,temporal_features0,temporal_features1,temporal_features2,temporal_features3,temporal_features4,temporal_features5,temporal_features6,temporal_features7,...,temporal_features214,temporal_features215,temporal_features216,temporal_features217,temporal_features218,temporal_features219,temporal_features220,temporal_features221,temporal_features222,temporal_features223
246,0.750180,234.501,0.208941,0.199116,0.165297,0.119200,0.228610,0.145534,0.215040,0.307783,...,-3.221379,9.918351,0.685443,0.43043,0.512893,0.06227,6.16508,6.10281,3.432451,16.253798
2135,0.779940,93.495,0.265258,0.622987,0.496264,0.125611,0.320337,0.070170,0.228531,0.134348,...,-0.242559,-0.207658,0.314515,0.28934,0.046540,0.06023,3.76739,3.70716,4.999788,63.627140
2533,0.519803,124.035,0.431988,0.444268,0.396486,0.235402,0.294019,0.271680,0.205089,0.219194,...,-1.116827,2.337863,0.312406,0.25916,0.018925,0.06444,1.52222,1.45778,1.978682,12.733705


In [30]:
y_pred = gb1.predict(x_test_3_songs) # get the predicted y-values
print(y_pred)

[0.04867972 0.51086412 0.83163573]


In [ ]:
# These danceability scores are pretty close!
# Model agreed that Wooden Ships is not very danceable with a score of 0.04867972 (actual: 0.05166771)
# Model agreed that Shakkei (Remixed) by Origamibiro is medium danceable with a score of 0.51086412 (actual 0.44688061)
# And Model agreed that Niris by Nicky Cook is super danceable with a score of 0.83163573 (actual 0.94879937)